In [1]:
import os
import random
import jpeglib
import tempfile
import numpy as np
from os import path
import pandas as pd
import seaborn as sns
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image

# set arbitrary/default jpeglib version
v_arbitrary = '9e'
jpeglib.version.set(v_arbitrary)

# set subsample size
N_samples = 10

# database path
db_path = Path.home() / 'Datasets'

### Load ALASKA Dataset 
We only need the Alaska Dataset, since only colored images produce differences.

In [2]:
# Load ALASKA2 dataset
alaska_path = db_path / 'ALASKA_v2_TIFF_256_COLOR'
alaska_names = [alaska_path / f for f in os.listdir(alaska_path)]
print("Loaded ALASKA2 database with", len(alaska_names), "images.")

# sample without replacement
random.seed(543)
alaska_names_sub = random.sample(alaska_names, N_samples-2)
#alaska_names_sub = alaska_names[:N_samples-2]

# add most and least saturated images to the sample set
alaska_names_sub.append((alaska_path / '10343.tif',98491)[0])
alaska_names_sub.append((alaska_path / '05887.tif', 78128)[0])

# save selection
import csv
# with open('quantification_files.csv', 'w') as fp:
#    csv.writer(fp).writerows([[str(f).replace('/home/nora', '~')] for f in alaska_names_sub])
# load selection
with open('quantification_files.csv') as fp:
    alaska_names_sub = [f[0].replace('~',str(Path.home())) for f in csv.reader(fp)]
    
# load the image with PIL
alaska = np.array([plt.imread(f) for f in alaska_names_sub])
print("Input shape", alaska.shape)

Loaded ALASKA2 database with 80004 images.
Input shape (1000, 256, 256, 3)


## Decompression VERSIONS experiments

For quantification of decompression mismatch we use PSNR.

In [9]:
def decompress_versions(**kwargs):
    tmp = tempfile.NamedTemporaryFile() # create temporary file
    psnr = []
    for i in range(alaska.shape[0]):
        # compress with arbitrary
        with jpeglib.version(v_arbitrary):
            if "sf" in kwargs:
                jpeglib.JPEG().write_spatial(tmp.name, alaska[i], samp_factor=kwargs["sf"])
            else:
                jpeglib.JPEG().write_spatial(tmp.name, alaska[i])
                
        # decompress with each version
        with jpeglib.version(kwargs["v1"]):
            if "dct" in kwargs:
                x_v1 = jpeglib.JPEG(tmp.name).read_spatial(dct_method=kwargs["dct"])
            else:
                x_v1 = jpeglib.JPEG(tmp.name).read_spatial()
        with jpeglib.version(kwargs["v2"]):
            if "dct" in kwargs:
                x_v2 = jpeglib.JPEG(tmp.name).read_spatial(dct_method=kwargs["dct"])
            else:
                x_v2 = jpeglib.JPEG(tmp.name).read_spatial()
        # compute psnr
        psnr.append(PSNR(x_v1, x_v2))
    return psnr

In [4]:
def decompress_parameters(**kwargs):
    tmp = tempfile.NamedTemporaryFile() # create temporary file
    # iterate alaska subsample
    psnr = []
    for i in range(alaska.shape[0]):
        with jpeglib.version(v_arbitrary):
            # compress with arbitrary
            if "qf1" in kwargs:
                jpeglib.JPEG().write_spatial(tmp.name, alaska[i], qt=kwargs["qf1"])
                x1 = jpeglib.JPEG(tmp.name).read_spatial()
                
                jpeglib.JPEG().write_spatial(tmp.name, alaska[i], qt=kwargs["qf2"])
                x2 = jpeglib.JPEG(tmp.name).read_spatial()
            else:
                jpeglib.JPEG().write_spatial(tmp.name, alaska[i])
                
            # decompress with arbitrary
            if "dct1" in kwargs:
                x1 = jpeglib.JPEG(tmp.name).read_spatial(dct_method=kwargs["dct1"])
                x2 = jpeglib.JPEG(tmp.name).read_spatial(dct_method=kwargs["dct2"])
            
            if "flag1" in kwargs:
                x1 = jpeglib.JPEG(tmp.name).read_spatial(flags=[kwargs["flag1"]])
                x2 = jpeglib.JPEG(tmp.name).read_spatial(flags=[kwargs["flag2"]])
        # compute psnr
        psnr.append(PSNR(x1, x2))
    
    return psnr

In [10]:
def PSNR(x_ref, x_noisy):
    """PSNR of two signals."""
    D = x_ref.astype(np.float32) - x_noisy.astype(np.float32)
    mse = (D**2).mean()
    maxi = 255#x_ref.max().astype(np.float64)
    return np.log10(maxi**2/mse)*10


def get_mismatching_images_decomp(psnr):
    psrn = np.array(psnr)
    return (~np.isinf(psrn)).sum()

def get_quantile_decomp(psnr):
    psnr = np.array(psnr)
    psnr = psnr[~np.isinf(psnr)]
    median = np.median(psnr)
    q5 = np.quantile(psnr, .05)    
    q95 = np.quantile(psnr, .95)
    return median, q5, q95

def print_eval_decomp_versions(**comp):
    print(comp["v1"], " vs. ", comp["v2"])
    psnr = decompress_versions(**comp)
    print(get_mismatching_images_decomp(psnr), "/", alaska.shape[0], "mismatching images")
    if "dct" in comp:
        print("DCT: ", comp["dct"])
    if "sf" in comp:
        print("SF: ", comp["sf"])
    
    if psnr:
        median, q5, q95 = get_quantile_decomp(psnr) 
    print("median: ", median, " q5: ", q5, " q95: ", q95 )
    print(f"{comp['v1']} vs {comp['v2']} & ${get_mismatching_images_decomp(psnr)}$ & ${round(q5, 2)}$ & ${round(median, 2)}$ & ${round(q95, 2)}$ \\")


def print_eval_decomp_para(**comp):
    if "dct1" in comp:
        print("DCT: ", comp["dct1"], " vs. ", comp["dct2"])
    if "qf1" in comp:
        print("QF: ", comp["qf1"], " vs. ", comp["qf2"])
    if "flag1" in comp:
        print("FLAG: ",comp["flag1"], " vs. ", comp["flag2"])
        
    psnr = decompress_parameters(**comp)
    
    print(get_mismatching_images_decomp(psnr), "/", alaska.shape[0], "mismatching images")
    if psnr:
        median, q5, q95 = get_quantile_decomp(psnr) 
    print("median: ", median, " q5: ", q5, " q95: ", q95 )
    print(f"& ${get_mismatching_images_decomp(psnr)}$ & ${round(q5, 2)}$ & ${round(median, 2)}$ & ${round(q95, 2)}$ \\")

In [11]:
decomp_versions=[{"v1":"6b", "v2":"7"},
                 {"v1":"9", "v2":"9a"},
                {"v1":"6b", "v2":"9a"}]

decomp_versions_sf = [{ "v1":"6b",
                    "v2":"turbo210",
                    "sf": ((1,2),(1,1),(1,1))},
                  { "v1":"turbo210",
                    "v2":"7",
                    "sf": ((1,2),(1,1),(1,1))},
                  { "v1":"9",
                    "v2":"9a",
                    "sf": ((1,2),(1,1),(1,1))}]

decomp_versions_sf_dct = [{"v1":"8",
                       "v2":"8a",
                       "sf": ((2,2),(1,1),(1,1)),
                       "dct": 'JDCT_FLOAT'},
                      {"v1":"9a",
                       "v2":"9b",
                       "sf": ((2,2),(1,1),(1,1)),
                       "dct": 'JDCT_FLOAT'},
                      {"v1":"9a",
                       "v2":"9b",
                       "sf": ((2,2),(1,1),(1,1)),
                       "dct": 'JDCT_IFAST'}]
all_decomp_versions = [decomp_versions, decomp_versions_sf, decomp_versions_sf_dct]


decomp_para_dct = [{"v":"9e",
              "dct1":"JDCT_ISLOW",
              "dct2":"JDCT_IFAST"},
                   {"v":"9e",
              "dct1":"JDCT_ISLOW",
              "dct2":"JDCT_FLOAT"},
                  {"v":"9e",
              "dct1":"JDCT_IFAST",
              "dct2":"JDCT_FLOAT"}]
decomp_para_qf = [{"v":"9e",
              "qf1":"75",
              "qf2":"90"},
            {"v":"9e",
              "qf1":"90",
              "qf2":"95"},
                 {"v":"9e",
              "qf1":"95",
              "qf2":"100"}]
decomp_para_flag = [{"v":"9e",
              "flag1":"-DO_FANCY_UPSAMPLING",
              "flag2":"+DO_FANCY_UPSAMPLING"}]

# all_decomp_para = [decomp_para_dct, decomp_para_qf, decomp_para_flag]
all_decomp_para = [decomp_para_dct, decomp_para_qf]


In [17]:
print("DECOMPRESSION VERSIONS")
for comparisons in all_decomp_versions:
    print("-----------------------------------------------")
    for comp in comparisons:
        print_eval_decomp_versions(**comp)
        print("\n")
   

DECOMPRESSION VERSIONS
-----------------------------------------------
6b  vs.  7


<ipython-input-15-1b011ea693a2>:6: RuntimeWarning: divide by zero encountered in double_scalars
  return np.log10(maxi**2/mse)*10


992 / 1000 mismatching images
median:  48.5536947017925  q5:  39.57174655086412  q95:  58.09277975835018
6b vs 7 & $992$ & $39.57$ & $48.55$ & $58.09$ \


9  vs.  9a
248 / 1000 mismatching images
median:  92.03591546276344  q5:  80.24493521484736  q95:  101.06681533268286
9 vs 9a & $248$ & $80.24$ & $92.04$ & $101.07$ \


6b  vs.  9a
992 / 1000 mismatching images
median:  48.5536947017925  q5:  39.57177050364372  q95:  58.09277975835018
6b vs 9a & $992$ & $39.57$ & $48.55$ & $58.09$ \


-----------------------------------------------
6b  vs.  turbo210
993 / 1000 mismatching images
SF:  ((1, 2), (1, 1), (1, 1))
median:  49.139036926873715  q5:  38.481113775892  q95:  60.29469493120001
6b vs turbo210 & $993$ & $38.48$ & $49.14$ & $60.29$ \


turbo210  vs.  7
993 / 1000 mismatching images
SF:  ((1, 2), (1, 1), (1, 1))
median:  50.89464522101735  q5:  41.23828832846785  q95:  61.29701105781035
turbo210  vs  7  & $ 50.89 $ & $ 993 $ & $ 41.24 $, $ 61.3 $ \


9  vs.  9a
280 / 1000 mismatchin

In [8]:
 print("\n")
    
print("\n")
print("DECOMPRESSION PARAMETERS")
for comparisons in all_decomp_para:
    print("-----------------------------------------------")
    for comp in comparisons:
        print_eval_decomp_para(**comp)
        print("\n")
    print("\n")





DECOMPRESSION PARAMETERS
-----------------------------------------------
DCT:  JDCT_ISLOW  vs.  JDCT_IFAST


/var/folders/l7/630hr2nx5d3b40lh2zrf65w80000gn/T/ipykernel_21319/293968784.py:6: RuntimeWarning: divide by zero encountered in true_divide
  return np.log10(maxi**2/mse)*10


999 / 1000 mismatching images
median:  58.83939070184671  q5:  56.147529006218335  q95:  63.06159198766774
 & $ 999 $ & $ 56.15 $ & $ 58.84 $, $ 63.06 $ \


DCT:  JDCT_ISLOW  vs.  JDCT_FLOAT
999 / 1000 mismatching images
median:  66.88546047786039  q5:  65.65314780381381  q95:  69.97102999306885
 & $ 999 $ & $ 65.65 $ & $ 66.89 $, $ 69.97 $ \


DCT:  JDCT_IFAST  vs.  JDCT_FLOAT
999 / 1000 mismatching images
median:  58.9149357671414  q5:  56.202810140126765  q95:  63.21615599900598
 & $ 999 $ & $ 56.2 $ & $ 58.91 $, $ 63.22 $ \




-----------------------------------------------
QF:  75  vs.  90
1000 / 1000 mismatching images
median:  34.88032295018609  q5:  27.77764772049858  q95:  42.67130199023175
 & $ 1000 $ & $ 27.78 $ & $ 34.88 $, $ 42.67 $ \


QF:  90  vs.  95
1000 / 1000 mismatching images
median:  36.6045491261398  q5:  30.316648415336537  q95:  44.764280872755464
 & $ 1000 $ & $ 30.32 $ & $ 36.6 $, $ 44.76 $ \






For comparison, we look at PSNR between different DCT methods - islow and ifast.

## COMPRESSION

For quantification of compression mismatch we use % of match in DCT.

In [9]:
def DCT_match_nz(dct1, dct2):
    (Y1,CbCr1),(Y2,CbCr2) = dct1,dct2
    dY = (Y1[Y1 != 0] == Y2[Y1 != 0]).sum()
    dCbCr = (CbCr1[CbCr1 != 0] == CbCr2[CbCr1 != 0]).sum()
    # % of matched DCT coefficients (Y + CbCr)
    return (dY + dCbCr) / ((Y1 != 0).sum() + (CbCr1 != 0).sum())

def DCT_match_log(dct1, dct2):
    (Y1,CbCr1),(Y2,CbCr2) = dct1,dct2
    dY = (Y1 != Y2).sum()
    dCbCr = (CbCr1 != CbCr2).sum()
    # % of matched DCT coefficients (Y + CbCr)
    match_pct =  (dY + dCbCr) / (Y1.size + CbCr1.size) 
    return np.log10(match_pct)

def get_missing_img_comp(match):
    match = np.array(match)
    return np.isnan(match)

def get_mismatching_img_comp(match):
    return (~np.isinf(match))

def get_quantile_comp(match):
    return np.quantile(match, [.5, .05,.95])

def print_evaluation_comp_version(**comp):
    print(comp["v1"], " vs. ", comp["v2"])
    
    nz_match, log_match = compression_versions(**comp)
    matches = [nz_match, log_match]
    
    print_var = ["NZ", "LOG"]
    for i, match in enumerate(matches):
        if match:
            median, q5, q95 = get_quantile_comp(match) 
        print(print_var[i])
        print(get_missing_img_comp(match).sum(), "/", alaska.shape[0], "missing images")
        print(get_mismatching_img_comp(match).sum(), "/", alaska.shape[0], "mismatching images")
        print("median: ", median, " q5: ", q5, " q95: ", q95 )
        print(f"{comp['v1']} vs {comp['v2']} & ${get_mismatching_img_comp(match).sum()}$ & ${round(q5, 2)}$ & ${round(median, 2)}$ & ${round(q95, 2)}$ \\")

        
def print_evaluation_comp_para(**comp):
    if "dct1" in comp:
        print("DCT: ", comp["dct1"], " vs. ", comp["dct2"])
    if "qf1" in comp:
        print("QF: ", comp["qf1"], " vs. ", comp["qf2"])
    if "flag1" in comp:
        print("FLAG: ",comp["flag1"], " vs. ", comp["flag2"] if comp["flag2"] is not None else "<empty>")
    
    nz_match, log_match = compression_parameters(**comp)
    matches = [nz_match, log_match]
    
    print_var = ["NZ", "LOG"]
    for i, match in enumerate(matches):
        if match:
            median, q5, q95 = get_quantile_comp(match) 
        print(print_var[i])
        print(get_missing_img_comp(match).sum(), "/", alaska.shape[0], "missing images")
        print(get_mismatching_img_comp(match).sum(), "/", alaska.shape[0], "mismatching images")
        print(" q5: ", q5, "median: ", median, " q95: ", q95 )
        print(f"& ${get_mismatching_img_comp(match).sum()}$ & ${round(q5, 2)}$ & ${round(median, 2)}$& ${round(q95, 2)}$ \\")
        
               
        

In [10]:
def compression_versions(**kwargs):
    tmp = tempfile.NamedTemporaryFile()
    nz_match,log_match = [],[]
    for i in range(alaska.shape[0]):
        # compress with version 1
        with jpeglib.version(kwargs["v1"]):
            jpeglib.JPEG().write_spatial(tmp.name, alaska[i])
        with jpeglib.version(kwargs["v1"]):
            Y_v1,CbCr_v1,_ = jpeglib.JPEG(tmp.name).read_dct()
            
        # compress with version 2
        with jpeglib.version(kwargs["v2"]):
            jpeglib.JPEG().write_spatial(tmp.name, alaska[i])
        with jpeglib.version(kwargs["v2"]):
            Y_v2,CbCr_v2,_ = jpeglib.JPEG(tmp.name).read_dct()
        
        # compute nz match
        nz_match.append(DCT_match_nz((Y_v1,CbCr_v1), (Y_v2,CbCr_v2)))
        log_match.append(DCT_match_log((Y_v1,CbCr_v1), (Y_v2,CbCr_v2)))
    
    return nz_match,log_match

In [11]:
def compression_parameters(**kwargs):
    tmp = tempfile.NamedTemporaryFile()
    nz_match,log_match = [],[]
    for i in range(alaska.shape[0]):
        # compress with version
        with jpeglib.version(kwargs["v"]):
            if "qf1" in kwargs:
                jpeglib.JPEG().write_spatial(tmp.name, alaska[i], qt=kwargs["qf1"])
                Y_v1,CbCr_v1,_ = jpeglib.JPEG(tmp.name).read_dct()
                jpeglib.JPEG().write_spatial(tmp.name, alaska[i], qt=kwargs["qf2"])
                Y_v2,CbCr_v2,_ = jpeglib.JPEG(tmp.name).read_dct()
                
            if "dct1" in kwargs:
                jpeglib.JPEG().write_spatial(tmp.name, alaska[i], dct_method=kwargs["dct1"])
                Y_v1,CbCr_v1,_ = jpeglib.JPEG(tmp.name).read_dct()
                jpeglib.JPEG().write_spatial(tmp.name, alaska[i], dct_method=kwargs["dct2"])
                Y_v2,CbCr_v2,_ = jpeglib.JPEG(tmp.name).read_dct()
            
            if "flag1" in kwargs:
                flag1 = [kwargs["flag1"]] if kwargs["flag1"] is not None else []
                flag2 = [kwargs["flag2"]] if kwargs["flag2"] is not None else []
                jpeglib.JPEG().write_spatial(tmp.name, alaska[i], flags=flag1, samp_factor=((1,2),(1,1),(1,1)))
                Y_v1,CbCr_v1,_ = jpeglib.JPEG(tmp.name).read_dct()
                jpeglib.JPEG().write_spatial(tmp.name, alaska[i], flags=flag2, samp_factor=((1,2),(1,1),(1,1)))
                Y_v2,CbCr_v2,_ = jpeglib.JPEG(tmp.name).read_dct()
                
        
       
        # compute nz match
        nz_match.append(DCT_match_nz((Y_v1,CbCr_v1), (Y_v2,CbCr_v2)))
        log_match.append(DCT_match_log((Y_v1,CbCr_v1), (Y_v2,CbCr_v2)))
    
    return nz_match,log_match


In [12]:
#per image : log10(share of all mismatching DCT coefficients)

In [13]:
comp_versions=[
    {
        "v1":"6b",
        "v2":"7"},
    {
        "v1":"9d",
        "v2":"9e"},
    {
        "v1":"6b",
        "v2":"9e"}]

comp_para_dct = [
{
    "v":"9e",
    "dct1":"JDCT_ISLOW",
    "dct2":"JDCT_FLOAT"    
},
{
    "v":"9e",
    "dct1":"JDCT_ISLOW",
    "dct2":"JDCT_IFAST"    
}]
comp_para_qf = [{"v":"9e",
              "qf1":"75",
              "qf2":"90"},
            {"v":"9e",
              "qf1":"90",
              "qf2":"95"},
               {"v":"9e",
              "qf1":"95",
              "qf2":"100"}]
comp_para_flag = [{"v":"8",
              "flag1":"DO_FANCY_UPSAMPLING",
              "flag2":None}]#"+DO_FANCY_UPSAMPLING"}]

all_para_compression = [comp_para_dct, comp_para_qf, comp_para_flag]
#all_para_compression = [ comp_para_qf, comp_para_flag]
all_versions_compression = [comp_versions]


In [25]:
print("COMPRESSION VERSIONS")
for comparisons in all_versions_compression:
    print("-----------------------------------------------")
    for comp in comparisons:
        print_evaluation_comp_version(**comp)
        print("\n")
    print("\n")
    
print("\n")

COMPRESSION VERSIONS
-----------------------------------------------
6b  vs.  7


/var/folders/l7/630hr2nx5d3b40lh2zrf65w80000gn/T/ipykernel_18684/2520111299.py:14: RuntimeWarning: divide by zero encountered in log10
  return np.log10(match_pct)


NZ
0 / 1000 missing images
1000 / 1000 mismatching images
median:  0.996886277611759  q5:  0.9929380979820672  q95:  0.9987583524620162
6b vs 7 & $1000$ & $0.99$ & $1.0$ & $1.0$ \
LOG
0 / 1000 missing images
995 / 1000 mismatching images
median:  -3.078757337295664  q5:  -3.6308433536617875  q95:  -2.6086662783180548
6b vs 7 & $995$ & $-3.63$ & $-3.08$ & $-2.61$ \


9d  vs.  9e
NZ
0 / 1000 missing images
1000 / 1000 mismatching images
median:  0.9796591883448269  q5:  0.9359063240833658  q95:  0.9933288526340704
9d vs 9e & $1000$ & $0.94$ & $0.98$ & $0.99$ \
LOG
0 / 1000 missing images
995 / 1000 mismatching images
median:  -2.474057249801493  q5:  -3.0194433360796817  q95:  -2.2833012287035497
9d vs 9e & $995$ & $-3.02$ & $-2.47$ & $-2.28$ \


6b  vs.  9e
NZ
0 / 1000 missing images
1000 / 1000 mismatching images
median:  0.9769723860790598  q5:  0.9324504272541915  q95:  0.9922410991359658
6b vs 9e & $1000$ & $0.93$ & $0.98$ & $0.99$ \
LOG
0 / 1000 missing images
995 / 1000 mismatchin

In [15]:

print("COMPRESSION PARAMETERS")
for comparisons in all_para_compression:
    print("-----------------------------------------------")
    for comp in comparisons:
        print_evaluation_comp_para(**comp)
        print("\n")
    print("\n")

COMPRESSION PARAMETERS
-----------------------------------------------
DCT:  JDCT_ISLOW  vs.  JDCT_FLOAT
NZ
0 / 1000 missing images
1000 / 1000 mismatching images
 q5:  0.9842197522471237 median:  0.9896980705289418  q95:  0.9928846279357652
 & $ 1000 $ & $ 0.99 $ & $ 0.98 $, $ 0.99 $ \
LOG
0 / 1000 missing images
1000 / 1000 mismatching images
 q5:  -3.1007567808648 median:  -2.8022394915090887  q95:  -2.6766008442224627
 & $ 1000 $ & $ -2.8 $ & $ -3.1 $, $ -2.68 $ \


DCT:  JDCT_ISLOW  vs.  JDCT_IFAST
NZ
0 / 1000 missing images
1000 / 1000 mismatching images
 q5:  0.949619459302893 median:  0.962532380365299  q95:  0.9719488738869432
 & $ 1000 $ & $ 0.96 $ & $ 0.95 $, $ 0.97 $ \
LOG
0 / 1000 missing images
1000 / 1000 mismatching images
 q5:  -2.396029137742964 median:  -2.153093142305182  q95:  -2.0075043328713496
 & $ 1000 $ & $ -2.15 $ & $ -2.4 $, $ -2.01 $ \




-----------------------------------------------
QF:  75  vs.  90
NZ
0 / 1000 missing images
1000 / 1000 mismatching ima

/var/folders/l7/630hr2nx5d3b40lh2zrf65w80000gn/T/ipykernel_18684/2520111299.py:14: RuntimeWarning: divide by zero encountered in log10
  return np.log10(match_pct)


NZ
0 / 1000 missing images
1000 / 1000 mismatching images
 q5:  1.0 median:  1.0  q95:  1.0
 & $ 1000 $ & $ 1.0 $ & $ 1.0 $, $ 1.0 $ \
LOG
0 / 1000 missing images
0 / 1000 mismatching images
 q5:  -inf median:  -inf  q95:  -inf
 & $ 0 $ & $ -inf $ & $ -inf $, $ -inf $ \






## Old experiments

In [ ]:
# helper functions for experiments
def compression_test(versions, flag):
    images_rgb = {'version': [], 'image': [], 'dct': []}
    kw = {'qt': 75, 'in_color_space': 'JCS_RGB', 'flags': [flag]}
    
    with tempfile.TemporaryDirectory() as tmp:
        for e, v_compress in enumerate(versions):
            
            # compress with each version
            fnames = [str(Path(tmp) / f'{i}.jpeg') for i in range(alaska.shape[0])]
            with jpeglib.version(v_compress):
                [jpeglib.JPEG().write_spatial(fname, alaska[i], **kw) for i,fname in enumerate(fnames)] 
            
            # decompress with single (arbitrary) version
            with jpeglib.version(v_arbitrary):
                images_rgb['version'].append(v_compress)
                images_rgb['image'].append(np.array([
                    jpeglib.JPEG(fname).read_spatial(flags=[flag]) for fname in fnames
                ]))
                images_rgb['dct'].append([
                    jpeglib.JPEG(fname).read_dct() for fname in fnames
                ])
                
    return pd.DataFrame(images_rgb)

def decompression_test(versions, flag):
    images_rgb = {'version': [], 'image': []}
    kw = {'qt': 75, 'in_color_space': 'JCS_RGB', 'flags': [flag]}
    
    with tempfile.TemporaryDirectory() as tmp:
        for e, v_decompress in enumerate(versions):
            
            # compress with single (arbitrary) version
            fnames = [str(Path(tmp) / f'{i}.jpeg') for i in range(alaska.shape[0])]
            with jpeglib.version(v_arbitrary):
                [jpeglib.JPEG().write_spatial(fname, alaska[i], **kw) for i,fname in enumerate(fnames)]
            
            # decompress with each version
            with jpeglib.version(v_decompress):
                images_rgb['version'].append(v_decompress)
                images_rgb['image'].append(np.array([
                    jpeglib.JPEG(fname).read_spatial(flags=[flag]) for fname in fnames
                ]))
                
    return pd.DataFrame(images_rgb)  

def fancy_upsampling_test(versions, flag):
    # sampling factor
    samp_factors = [
        ((1,1),(1,1),(1,1)), # 4:4:4
        ((1,2),(1,2),(1,2)),
        ((2,1),(2,1),(2,1)),

        ((1,2),(1,1),(1,1)), # 4:4:0
        ((2,2),(2,1),(2,1)),
        ((1,4),(1,2),(1,2)),
        ((1,2),(1,2),(1,1)),   # Cb 4:4:4 Cr 4:4:0
        ((1,2),(1,1),(1,2)),   # Cb 4:4:0 Cr 4:4:4

        ((2,1),(1,1),(1,1)), # 4:2:2
        ((2,2),(1,2),(1,2)),
        ((2,1),(2,1),(1,1)),   # Cb 4:4:4 Cr 4:2:2
        ((2,1),(1,1),(2,1)),   # Cb 4:2:2 Cr 4:4:4

        ((2,2),(1,1),(1,1)), # 4:2:0
        ((2,2),(2,1),(1,1)),   # Cb 4:4:0 Cr 4:2:0
        ((2,2),(1,1),(2,1)),   # Cb 4:2:0 Cr 4:4:0
        ((2,2),(1,2),(1,1)),   # Cb 4:2:2 Cr 4:2:0
        ((2,2),(1,1),(1,2)),   # Cb 4:2:0 Cr 4:2:2
        ((2,2),(2,2),(1,1)),   # Cb 4:4:4 Cr 4:2:0
        ((2,2),(2,2),(2,1)),   # Cb 4:4:4 Cr 4:4:0
        ((2,2),(2,2),(1,2)),   # Cb 4:4:4 Cr 4:2:2
        ((2,2),(1,1),(2,2)),   # Cb 4:2:0 Cr 4:4:4
        ((2,2),(2,1),(2,2)),   # Cb 4:4:0 Cr 4:4:4
        ((2,2),(1,2),(2,2)),   # Cb 4:2:2 Cr 4:4:4

        ((4,1),(1,1),(1,1)), # 4:1:1
        ((4,1),(2,1),(1,1)),   # Cb 4:2:2 Cr 4:1:1
        ((4,1),(1,1),(2,1)),   # Cb 4:1:1 Cr 4:2:2

        ((4,2),(1,1),(1,1)), # 4:1:0

        ((1,4),(1,1),(1,1)), # 1:0.5:0
        ((1,4),(1,2),(1,1)),

        ((2,4),(1,1),(1,1)), # 2:0.5:0

        ((3,1),(1,1),(1,1)), # 3:1:1
        ((3,1),(3,1),(1,1)),   # Cb 4:4:4 Cr 3:1:1
        ((3,1),(1,1),(3,1)),   # Cb 3:1:1 Cr 4:4:4
    ]

    images_rgb_cdFU = {'version': [], 'samp_factor': [], 'dct': [], 'image': []}
    kw = {'qt': 75, 'flags': [flag], 'in_color_space': 'JCS_RGB'}

    with tempfile.TemporaryDirectory() as tmp:
    
        # iterate versions
        for i,v_decompress in enumerate(versions):

            # iterate samp factors
            for samp_factor in samp_factors:

                # compress each image with version
                fnames = [str(Path(tmp) / f'{i}.jpeg') for i in range(alaska.shape[0])]
                with jpeglib.version(v_arbitrary):
                    [jpeglib.JPEG().write_spatial(fname, alaska[i], samp_factor=samp_factor, **kw)
                         for i,fname in enumerate(fnames)]

                # decompress with single (arbitrary) version
                with jpeglib.version(v_decompress):
                    images_rgb_cdFU['version'].append(v_decompress)
                    images_rgb_cdFU['samp_factor'].append(samp_factor)
                    images_rgb_cdFU['image'].append(np.array([
                        jpeglib.JPEG(fname).read_spatial(flags=['DO_FANCY_UPSAMPLING']) for fname in fnames
                    ]))
                    images_rgb_cdFU['dct'].append([
                        jpeglib.JPEG(fname).read_dct() for fname in fnames
                    ])

    return pd.DataFrame(images_rgb_cdFU)


# Vizualisation of the differences

Here you can see how much the values differ for all combinations of produced clusters per experiment.

We vizualize 13 histograms in total:
### Baseline Compression
- (6b + 7), (6b + 9e), (7 + 9e)

### Baseline Decompression
- (6b + 7), (6b + 9a), (7 + 9a)

### Simple Upscaling
- (6b + 9a)

### Fancy Upsampling
- (6b + turbo), (6b + 7), (6b + 9a), (turbo210 + 7), (turbo210 + 9a), (7 + 9a)

In [ ]:
# Helper Functions Vizualisation

def get_images_of_versions(images, versions):
    (imageV1,) = images[images.version == versions[0]].image
    (imageV2,) = images[images.version == versions[1]].image
    return (imageV1.astype(np.int32) - imageV2.astype(np.int32))

def plot_histogram(D, version, title):
    fig,ax = plt.subplots(1,1, figsize=[12,7], sharey=True)
    #fig.subtitle(f'Distribution of differences for {versions}', fontsize=11)
    
    sns.set_theme()
    sns.despine(left=False, bottom=False)
    g = sns.histplot(D.flatten(), ax=ax, binwidth=1., stat='density')
    g.set_yscale('log');
    plot_path = path.join('..', 'text', 'forensic', 'figures', f'histogram_{title}_{version}.png')
    plt.savefig(plot_path);

def compare_and_plot_subplots(images, versions, title):
    fig,axs = plt.subplots(1,len(versions), figsize=(16,4), sharey=True)
    #fig.title(f'Distribution of differences for {title}', fontsize=14)
    
    for k, v1 in enumerate(versions):
        for v2 in versions[k:]:
            if v1 == v2:
                continue
            D = get_images_of_versions(images, [v1, v2])
            
            p = sns.histplot(D.flatten(), binwidth=1, ax=axs[k])
            p.set_title(f'{v1} - {v2}', fontsize=8)
            plot_path = path.join('..', 'text', 'forensic', 'figures', f'histogram_{title}_{v1}_{v2}.png')
            print("saving plot for ", title, " ", v1, " - ", v2)
            plt.savefig(plot_path);

## Baseline Compression

In [ ]:
versions = ['6b','7', '9e']
title = "BaselineCompression"
flag = 'DO_FANCY_UPSAMPLING'
images = compression_test(versions, flag)

# generate plot for all unique version combinations
# compare_and_plot_subplots(images, versions, title)
plot_histogram(images, ['6b', '9a'], title)

## Baseline Decompression

In [ ]:
versions = ['6b','7', '9a']
title = "BaselineDecompression"
flag = 'DO_FANCY_UPSAMPLING'
images = decompression_test(versions, flag)

# generate plot for all unique version combinations
compare_and_plot_subplots(images, versions, title)

## Fancy Upsampling

### Factor 4:4:0

In [ ]:
kw = {'qt': 75, 'flags': [], 'in_color_space': 'JCS_RGB', 'samp_factor': ((1,2),(1,1),(1,1))}
tmp = tempfile.TemporaryDirectory()
    
# compress image
fnames = [str(Path(tmp.name) / f'{i}.jpeg') for i in range(alaska.shape[0])]
with jpeglib.version(v_arbitrary):
    [jpeglib.JPEG().write_spatial(fname, alaska[i], **kw) for i,fname in enumerate(fnames)]

# decompress with each version
with jpeglib.version('6b'):
    x_6b = np.array([jpeglib.JPEG(fname).read_spatial(flags=['DO_FANCY_UPSAMPLING']) for fname in fnames])
with jpeglib.version('turbo210'):
    x_turbo = np.array([jpeglib.JPEG(fname).read_spatial(flags=['DO_FANCY_UPSAMPLING']) for fname in fnames])

# compute the difference
D_fu440_6b_turbo = x_6b.astype(np.int32) - x_turbo.astype(np.int32)

### Factor 4:2:2

In [ ]:
kw = {'qt': 75, 'flags': [], 'in_color_space': 'JCS_RGB', 'samp_factor': ((2,1),(1,1),(1,1))}
tmp = tempfile.TemporaryDirectory()
    
# compress image
fnames = [str(Path(tmp.name) / f'{i}.jpeg') for i in range(alaska.shape[0])]
with jpeglib.version(v_arbitrary):
    [jpeglib.JPEG().write_spatial(fname, alaska[i], **kw) for i,fname in enumerate(fnames)]

# decompress with each version
with jpeglib.version('6b'):
    x_6b = np.array([jpeglib.JPEG(fname).read_spatial(flags=['DO_FANCY_UPSAMPLING']) for fname in fnames])
with jpeglib.version('7'):
    x_7 = np.array([jpeglib.JPEG(fname).read_spatial(flags=['DO_FANCY_UPSAMPLING']) for fname in fnames])

# compute the difference
D_fu422_6b_7 = x_6b.astype(np.int32) - x_turbo.astype(np.int32)

In [ ]:
# produce plot
#fig,ax = plt.subplots(1,1, figsize=[12,7], sharey=True)
#sns.set_theme()
#sns.despine(left=False, bottom=False)
#g = sns.histplot(D_440_6b_turbo.flatten(), ax=ax, binwidth=1., stat='density')
#g.set_yscale('log');
#g.set_xlabel('Difference')
#plot_path = path.join('..', 'text', 'forensic', 'figures', 'histogram_4:4:0_6b-turbo.png')
#plt.savefig(plot_path);

## DCT method

To compare the differences produced by differing versions, we now look at the differences, that different DCT methods produce.

In [ ]:
kw = {'qt': 75, 'flags': [], 'in_color_space': 'JCS_RGB', 'flags': ['DO_FANCY_UPSAMPLING']}
tmp = tempfile.TemporaryDirectory()
    
# compress image with islow
fnames = [str(Path(tmp.name) / f'{i}.jpeg') for i in range(alaska.shape[0])]
[jpeglib.JPEG().write_spatial(fname, alaska[i], dct_method='JDCT_ISLOW', **kw) for i,fname in enumerate(fnames)]
x_islow = np.array([jpeglib.JPEG(fname).read_spatial(flags=['DO_FANCY_UPSAMPLING']) for fname in fnames])

# compress image with ifast
[jpeglib.JPEG().write_spatial(fname, alaska[i], dct_method='JDCT_IFAST', **kw) for i,fname in enumerate(fnames)]
x_ifast = np.array([jpeglib.JPEG(fname).read_spatial(flags=['DO_FANCY_UPSAMPLING']) for fname in fnames])

# compute difference
D_islow_ifast = x_islow.astype(np.int32) - x_ifast.astype(np.int32)

## Violin plot

At last we create a violin plot presenting the complete picture, compared to different DCT methods.

In [ ]:
df = pd.concat([
    pd.DataFrame({
        'Scenario': '6b-turbo\nfancy upsampling 4:4:0',
        'Difference': D_fu440_6b_turbo[D_fu440_6b_turbo != 0].flatten()
    }),
    pd.DataFrame({
        'Scenario': '6b-7\nfancy upsampling 4:2:2',
        'Difference': D_fu422_6b_7[D_fu422_6b_7 != 0].flatten()
    }),
    pd.DataFrame({
        'Scenario': 'islow-ifast',
        'Difference': D_islow_ifast[D_islow_ifast != 0].flatten()
    })
])
fig,ax = plt.subplots(1,1, figsize=[12,7], sharey=True)
sns.violinplot(x='Scenario', y='Difference', bw=.12, data=df, ax=ax, cut=0)
plt.ylim(-17, 17);

# PSNR

In [ ]:
def PSNR(x_ref, x_noisy):
    D = x_ref.astype(np.float32) - x_noisy.astype(np.float32)
    mse = (D**2).mean(axis=tuple(range(1,len(x_ref.shape))))
    maxi = x_ref.max(axis=tuple(range(1,len(x_ref.shape)))).astype(np.float64)
    return np.log10(maxi**2/mse)*10

In [ ]:
alaska.shape

### 6b-7 compression 4:2:2

In [ ]:
from scipy import stats

tmp = tempfile.NamedTemporaryFile() # create temporary file
for v in ['6b','7']:
    x_fu_422 = []
    # recompress
    for i in range(alaska.shape[0]):
        with jpeglib.version(v):
            jpeglib.JPEG().write_spatial(
                tmp.name, alaska[i],
                dct_method='JDCT_ISLOW', flags=['DO_FANCY_UPSAMPLING'], samp_factor=((2,1),(1,1),(1,1))
            )
        with jpeglib.version('6b'):
            x = jpeglib.JPEG(tmp.name).read_spatial(dct_method='JDCT_ISLOW')
        x_fu_422.append(x)

    # substract mode
    #Y_fu_422 = np.array(Y_fu_422) - stats.mode(Y_fu_422, axis=0)[0]
    #CbCr_fu_422 = np.array(CbCr_fu_422) - stats.mode(CbCr_fu_422, axis=0)[0]
    # compute psnr
    #psnr_Y_fu_422 = PSNR(alaska, np.array(Y_fu_422))
    #psnr_CbCr_fu_422 = PSNR(alaska, np.array(CbCr_fu_422))
    #print(psnr_Y_fu_422.shape, psnr_CbCr_fu_422.shape)
    psnr_fu_422 = PSNR(alaska, np.array(x_fu_422))
    print("6b-7 compression", v, psnr_fu_422.mean(), psnr_fu_422.std())

### 6b-7 decompression 4:2:2

In [ ]:
tmp = tempfile.NamedTemporaryFile() # create temporary file
for v in ['6b','7']:
    x_fu_422 = []
    for i in range(alaska.shape[0]):
        with jpeglib.version('6b'):
            jpeglib.JPEG().write_spatial(tmp.name, alaska[i],
                                         dct_method='JDCT_ISLOW',
                                         flags=['DO_FANCY_UPSAMPLING'],
                                         samp_factor=((2,1),(1,1),(1,1)))
        with jpeglib.version(v):
            x = jpeglib.JPEG(tmp.name).read_spatial(dct_method='JDCT_ISLOW')
        x_fu_422.append(x)
    # compute psnr
    psnr_fu_422 = PSNR(alaska, np.array(x_fu_422))
    print("FU", "4:2:2", v, psnr_fu_422.mean(), psnr_fu_422.std())

### 6b-turbo decompression 4:4:0

In [ ]:
tmp = tempfile.NamedTemporaryFile() # create temporary file
for v in ['6b','turbo210']:
    x_fu_440 = []
    for i in range(alaska.shape[0]):
        with jpeglib.version('6b'):
            jpeglib.JPEG().write_spatial(tmp.name, alaska[i],
                                         dct_method='JDCT_ISLOW',
                                         flags=['DO_FANCY_UPSAMPLING'],
                                         samp_factor=((1,2),(1,1),(1,1)))
        with jpeglib.version(v):
            x = jpeglib.JPEG(tmp.name).read_spatial(dct_method='JDCT_ISLOW')
        x_fu_440.append(x)
    # compute psnr
    psnr_fu_440 = PSNR(alaska, np.array(x_fu_440))
    print("FU", "4:2:2", v, psnr_fu_440.mean(), psnr_fu_440.std())

### 9-9a decompression

In [ ]:
tmp = tempfile.NamedTemporaryFile() # create temporary file
for v in ['9','9a']:
    x_9_9a = []
    for i in range(alaska.shape[0]):
        with jpeglib.version('9'):
            jpeglib.JPEG().write_spatial(tmp.name, alaska[i],
                                         dct_method='JDCT_ISLOW',
                                         flags=['DO_FANCY_UPSAMPLING'],
                                         samp_factor=((1,1),(1,1),(1,1)))
        with jpeglib.version(v):
            x = jpeglib.JPEG(tmp.name).read_spatial()
        x_9_9a.append(x)
    # compute psnr
    psnr_9_9a = PSNR(alaska, np.array(x_9_9a))
    print("9-9a", v, psnr_9_9a.mean(), psnr_9_9a.std())

### 9d-9e compression

In [ ]:
from scipy import stats

tmp = tempfile.NamedTemporaryFile() # create temporary file
for v in ['9d','9e']:
    x_9d_9e = []
    # recompress
    for i in range(alaska.shape[0]):
        with jpeglib.version(v):
            jpeglib.JPEG().write_spatial(
                tmp.name, alaska[i],
                dct_method='JDCT_ISLOW', flags=['DO_FANCY_UPSAMPLING'], samp_factor=((2,1),(1,1),(1,1))
            )
        with jpeglib.version('9d'):
            x = jpeglib.JPEG(tmp.name).read_spatial(dct_method='JDCT_ISLOW')
        x_9d_9e.append(x)

    psnr_9d_9e = PSNR(alaska, np.array(x_9d_9e))
    print("9d-9e compression", v, psnr_9d_9e.mean(), psnr_9d_9e.std())

### Compression DCT method

In [ ]:
from scipy import stats

tmp = tempfile.NamedTemporaryFile() # create temporary file
for method in ['JDCT_ISLOW','JDCT_IFAST']:
    x_islow_ifast = []
    # recompress
    for i in range(alaska.shape[0]):
        with jpeglib.version('9d'):
            jpeglib.JPEG().write_spatial(
                tmp.name, alaska[i],
                dct_method='JDCT_ISLOW', flags=['DO_FANCY_UPSAMPLING'], samp_factor=((2,1),(1,1),(1,1))
            )
            x = jpeglib.JPEG(tmp.name).read_spatial(dct_method=method)
        x_islow_ifast.append(x)

    psnr_islow_ifast = PSNR(alaska, np.array(x_islow_ifast))
    print("islow-ifast compression", method, psnr_islow_ifast.mean(), psnr_islow_ifast.std())

### Decompression DCT method

In [ ]:
from scipy import stats

tmp = tempfile.NamedTemporaryFile() # create temporary file
for method in ['JDCT_ISLOW','JDCT_IFAST']:
    x_islow_ifast = []
    # recompress
    for i in range(alaska.shape[0]):
        with jpeglib.version('9d'):
            jpeglib.JPEG().write_spatial(
                tmp.name, alaska[i],
                dct_method=method, flags=['DO_FANCY_UPSAMPLING'], samp_factor=((2,1),(1,1),(1,1))
            )
            x = jpeglib.JPEG(tmp.name).read_spatial(dct_method='JDCT_ISLOW')
        x_islow_ifast.append(x)

    psnr_islow_ifast = PSNR(alaska, np.array(x_islow_ifast))
    print("islow-ifast decompression", method, psnr_islow_ifast.mean(), psnr_islow_ifast.std())

### DCT method

In [ ]:
from scipy import stats

tmp = tempfile.NamedTemporaryFile() # create temporary file
for method in ['JDCT_ISLOW','JDCT_IFAST']:
    x_islow_ifast = []
    # recompress
    for i in range(alaska.shape[0]):
        with jpeglib.version('9d'):
            jpeglib.JPEG().write_spatial(
                tmp.name, alaska[i],
                dct_method=method, flags=['DO_FANCY_UPSAMPLING'], samp_factor=((2,1),(1,1),(1,1))
            )
            x = jpeglib.JPEG(tmp.name).read_spatial(dct_method=method)
        x_islow_ifast.append(x)

    psnr_islow_ifast = PSNR(alaska, np.array(x_islow_ifast))
    print("islow-ifast", method, psnr_islow_ifast.mean(), psnr_islow_ifast.std())

### DCT method: islow

In [ ]:
# create temporary file
with tempfile.NamedTemporaryFile() as tmp:
    # iterate versions
    for v in ['6b', 'turbo210', '7', '9a', '9e']:
        with jpeglib.version(v):
            
            # compress ifast
            x_ifast = []
            for i in range(alaska.shape[0]):
                jpeglib.JPEG().write_spatial(tmp.name, alaska[i], dct_method='JDCT_IFAST', samp_factor=((1,1),(1,1),(1,1)))
                x = jpeglib.JPEG(tmp.name).read_spatial(dct_method='JDCT_IFAST')
                x_ifast.append(x)
            # compute psnr
            psnr_ifast = PSNR(alaska, np.array(x_ifast))
            print("ifast", v, psnr_ifast.mean(), psnr_ifast.std())
            
            # compress islow
            x_islow = []
            for i in range(alaska.shape[0]):
                jpeglib.JPEG().write_spatial(tmp.name, alaska[i], dct_method='JDCT_ISLOW', samp_factor=((1,1),(1,1),(1,1)))
                x = jpeglib.JPEG(tmp.name).read_spatial(dct_method='JDCT_ISLOW')
                x_islow.append(x)
            # compute psnr
            psnr_islow = PSNR(alaska, np.array(x_islow))
            print("islow", v, psnr_islow.mean(), psnr_islow.std())
            
            # compress FU 4:2:2
            x_fu_422 = []
            for i in range(alaska.shape[0]):
                jpeglib.JPEG().write_spatial(tmp.name, alaska[i], dct_method='JDCT_ISLOW', flags=['DO_FANCY_UPSAMPLING'], samp_factor=((2,1),(1,1),(1,1)))
                x = jpeglib.JPEG(tmp.name).read_spatial(dct_method='JDCT_ISLOW')
                x_fu_422.append(x)
            # compute psnr
            psnr_islow = PSNR(alaska, np.array(x_fu_422))
            print("FU4:2:2", v, psnr_islow.mean(), psnr_islow.std())
            
            # compress FU 4:4:0
            x_fu_440 = []
            for i in range(alaska.shape[0]):
                jpeglib.JPEG().write_spatial(tmp.name, alaska[i], dct_method='JDCT_ISLOW', flags=['DO_FANCY_UPSAMPLING'], samp_factor=((1,2),(1,1),(1,1)))
                x = jpeglib.JPEG(tmp.name).read_spatial(dct_method='JDCT_ISLOW')
                x_fu_440.append(x)
            # compute psnr
            psnr_islow = PSNR(alaska, np.array(x_fu_440))
            print("FU4:4:0", v, psnr_islow.mean(), psnr_islow.std())
            


In [ ]:
# psnr ifast
psnr_ifast = {v: PSNR(img, x_ifast[v]) for v in versions}
#psnr_ifast_6b = PSNR(img, x_ifast['6b'])
print('psnr ifast: ', psnr_ifast, '\n')

# psnr islow
#psnr_islow = PSNR(versions, img, x_islow)
#print('psnr islow: ', psnr_islow, '\n')

# psnr fancy upsampling 4:2:2
#psnr_fancy_us_422 = PSNR(versions, img, x_fancy_us_422)
#print('psnr fancy upsampling 4:2:2: ', psnr_fancy_us_422, '\n')

# psnr fancy upsampling 4:4:0
#psnr_fancy_us_440 = PSNR(versions, img, x_fancy_us_440)
#print('psnr fancy upsampling 4:4:0: ', psnr_fancy_us_440, '\n')



In [ ]:
def calc_PSNR(x_ref, x_noisy):
    D = x_ref.astype(np.float32) - x_noisy.astype(np.float32)
    mse = (D**2).mean(axis=(0,1,2))
    maxi = x_ref.max(axis=(0,1,2)).astype(np.float64)
    #print(mse)
    return np.log10(maxi**2/mse)*10

In [ ]:
# Methods
# random_img = random.randrange(61600)
# reference_image = os.path.join(alaska_path, f'{random_img}.tif})
ref_img = os.path.join(alaska_path, "58580.tif")
img = np.array(Image.open(ref_img, 'r'))
print('Shape: ', img.shape)
plt.imshow(img);

In [ ]:
def compress(versions, kw):
    img_comp = {}
    for v in versions:
        with jpeglib.version(v):
            jpeglib.JPEG().write_spatial(ref_img, img, **kw)
            x = np.array(jpeglib.JPEG(ref_img).read_spatial(flags=['DO_FANCY_UPSAMPLING']))
        img_comp[v] = x
    return img_comp

def decompress(versions, kw):
    img_decomp = {}
    for v in versions:
        with jpeglib.version(v):
            jpeglib.JPEG().write_spatial(ref_img, img, **kw) 
            x = jpeglib.JPEG(ref_img).read_spatial(flags=['DO_FANCY_UPSAMPLING'])
        img_decomp[v] = x
    return img_decomp
    

In [ ]:
versions = ['6b', 'turbo210', '7', '9a', '9e']

# compress ifast
kw = {'qt': 75, 'flags': [], 'in_color_space': 'JCS_RGB', 'dct_method': 'JDCT_IFAST','flags': ['DO_FANCY_UPSAMPLING']}
x_ifast = compress(versions, kw)

# compress islow
kw = {'qt': 75, 'flags': [], 'in_color_space': 'JCS_RGB', 'dct_method': 'JDCT_ISLOW','flags': ['DO_FANCY_UPSAMPLING']}
x_islow = compress(versions, kw)

# fancy upsampling 4:2:2
kw = {'qt': 75, 'flags': [], 'in_color_space': 'JCS_RGB', 'flags': ['DO_FANCY_UPSAMPLING'], 'samp_factor': ((2,1),(1,1),(1,1))}
x_fancy_us_422 = decompress(versions, kw)

# fancy upsampling 4:4:0
kw = {'qt': 75, 'flags': [], 'in_color_space': 'JCS_RGB', 'flags': ['DO_FANCY_UPSAMPLING'], 'samp_factor': ((1,2),(1,1),(1,1))}
x_fancy_us_440 = decompress(versions, kw)



In [ ]:
def get_psnr(versions, img, x):
    psnr = {}
    for v in versions:
        psnr[v] = calc_PSNR(img, x[v])
    return psnr

In [ ]:
# psnr ifast
psnr_ifast = get_psnr(versions, img, x_ifast)
print('psnr ifast: ', psnr_ifast, '\n')

# psnr islow
psnr_islow = get_psnr(versions, img, x_islow)
print('psnr islow: ', psnr_islow, '\n')

# psnr fancy upsampling 4:2:2
psnr_fancy_us_422 = get_psnr(versions, img, x_fancy_us_422)
print('psnr fancy upsampling 4:2:2: ', psnr_fancy_us_422, '\n')


# psnr fancy upsampling 4:4:0
psnr_fancy_us_440 = get_psnr(versions, img, x_fancy_us_440)
print('psnr fancy upsampling 4:4:0: ', psnr_fancy_us_440, '\n')

